In [2]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
from io import StringIO
import json
import os
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator
from rules_engine import rules_engine
!pwd

/Users/roellk/Desktop/HMDA/hmda-test-files/python


In [3]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependancies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

In [4]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))

In [5]:
lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints
lar_validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties)

In [6]:
#Set parameters for data creation
file_length = 5 #set number of rows in test file
first = True
lei = None

In [13]:
#Create a sample TS row
#set dummy values for TS row
def make_ts_row(file_length, lei):
    ts_row = OrderedDict()
    ts_row["record_id"]="1"
    ts_row["inst_name"]="Ficus Bank"
    ts_row["calendar_year"]= "2018"
    ts_row["calendar_quarter"]="4"
    ts_row["contact_name"]="Mr. Smug Pockets"
    ts_row["contact_tel"]="555-555-5555"
    ts_row["contact_email"]="pockets@ficus.com"
    ts_row["contact_street_address"]="1234 Ficus Lane"
    ts_row["office_city"]="Ficusville"
    ts_row["office_state"]="UT"
    ts_row["office_zip"]="84096"
    ts_row["federal_agency"]="9"
    ts_row["lar_entries"]= str(file_length)
    ts_row["tax_id"]="01-0123456"
    ts_row["lei"]=str(lei)
    return ts_row

In [14]:
def get_const_list():
    """Creates a list of constraints from the functions in the lar_constraints object."""
    constraints = [] 
    for func in dir(lar_const):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            constraints.append(func)
    return constraints
            
def constraints_loop(constraints=[], row=None, row_base=None):
    for const in constraints:
        row = apply_constraint(row, const)
        diff = get_diff(row, row_base)
    return row

def apply_constraint(row, func):
    """Applies all constraints in the constrains list and returns a LAR row in dictionary format."""
    row_start = row.copy()
    row = getattr(lar_const, func)(row) #apply constraint to row
    diff_1, diff_2 = get_diff(row, row_start)
    if len(diff_1) > 0:
        print(str(func))
        print(diff_1, "\n\n", diff_2)
    return row

def get_diff(row, row_base):
    """Checks the difference between an initial row and the row after constraints are applied"""
    initial_row = set(row_base.items()) #convert initial row to set
    changed_row = set(row.items()) #convert constrained row to set
    diff_1 = (changed_row - initial_row) #subtract row sets to show changes from constraint funcs
    diff_2 = (initial_row - changed_row)
    return diff_1, diff_2

def validation(row, ts_row):
    """"""
    lar_data = pd.DataFrame(row, index=[1])
    ts_data = pd.DataFrame(ts_row, index=[0])
    rules_check = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, 
                             counties=counties) #instantiate edits rules engine
    rules_check.load_lar_data(lar_data)
    rules_check.load_ts_data(ts_data)
    for func in dir(rules_check):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            #print("applying:", func)
            getattr(rules_check, func)()
    return rules_check.results



In [ ]:
for i in range(0, file_length): #loop over file length
    print("making new row\n\n")
    if lei:
        row = lar_gen.make_row(lei=lei) #generate new LEI. The same LEI must be used for each row
    else:
        row = lar_gen.make_row() #create new row
        
    lei = row["lei"] #copy LEI from previous row
    iters = 1 #start iteration count for checking diff time

    ts_row = make_ts_row(file_length, lei) #create dictionary of TS row fields
    stop = False
    first = True #flag for starting the LAR dataframe
    while stop == False:
        row_base = row.copy() #copy row to enable diff
        res = pd.DataFrame(validation(row, ts_row))
        if len(res[res.status=="failed"])<=0:
            stop = True
        else:
            print("\nstarting constraints iteration {iter}".format(iter=iters))
            row = constraints_loop(get_const_list(), row, row_base)
        
        #diff_1, diff_2 = get_diff(row, row_base) #get difference between row and base row
        
        #if len(diff_1) == 0:
            #results = validation(row)
            #res_df = pd.DataFrame(results)
            #if len(res_df[res_df.status=="failed"])>0:
            #    print("\n\n\nedit fails:", len(res_df[res_df.status=="failed"]))
            #    print(res_df[res_df.status=="failed"])
            #    stop = False
            #else:
            #    stop = True
            #stop = True
        iters+=1
    
    if first: #create first row of dataframe
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
        print("finished row\n")
    else: #add additional rows to dataframe
        #print("concating")
        print("finished row\n")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)
        
lar_frame.reset_index(inplace=True) #reset index
lar_frame.drop("index", inplace=True, axis=1) #drop additional index column
print(iters)

making new row



starting constraints iteration 1
v610_const
{('loan_purpose', '1')} 

 {('loan_purpose', '2')}
v612_const
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_5', '2'), ('app_eth_3', '11'), ('app_eth_2', '1'), ('app_eth_4', '14')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
v629_const
v630_const
{('co_app_eth_3', ''), ('co_app_eth_2', '')} 

 {('co_app_eth_3', '2'), ('co_app_eth_2', '1')}
v631_const
v632_const
{('co_app_eth_1', '3')} 

 {('co_app_eth_1', '5')}
v633_const
v634_const
v635_1_const
v635_2_const
{('app_race_2', '41'), ('app_race_3', '25')} 

 {('app_race_2', ''), ('app_race_3', '')}
v635_3_const
v635_4_const
{('app_race_3', '2'), ('app_race_1', '3'), ('app_race_5', '4'), ('app_race_2', '5')} 

 {('app_race_2', '41'), ('app_race_5', '22'), ('app_race_3', '25'), ('app_race_1', '26')}
v636_const
v637_const
v638_co

 {('aus_result_1', '11'), ('aus_1', '2')}
v704_const
v705_const

starting constraints iteration 2
{('app_date', 'NA')} 

 {('app_date', '20180119')}
v610_const
v612_const
v613_2_const
v613_3_const
v613_4_const
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_3', '14'), ('app_eth_4', '2'), ('app_eth_5', '13'), ('app_eth_2', '11')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')} 

 {('app_eth_3', '14'), ('app_eth_4', '2'), ('app_eth_5', '13'), ('app_eth_2', '11')}
v629_const
v630_const
{('co_app_eth_4', ''), ('co_app_eth_5', ''), ('co_app_eth_3', ''), ('co_app_eth_2', '')} 

 {('co_app_eth_5', '14'), ('co_app_eth_2', '12'), ('co_app_eth_3', '13'), ('co_app_eth_4', '1')}
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
{('app_race_4', '41'), ('app_race_5', '5')} 

 {('app_race_4', '4'), ('app_race_5', '1')}
v635_3_const
v635_4_const


v700_const
v701_const
v702_const
v703_const
v704_const
v705_const

starting constraints iteration 2
v610_const
v612_const
v613_2_const
v613_3_const
v613_4_const
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')} 

 {('app_eth_3', '12'), ('app_eth_5', '11'), ('app_eth_2', '13'), ('app_eth_4', '1')}
v628_const
v629_const
v630_const
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
{('app_race_2', ''), ('app_race_3', ''), ('app_race_4', ''), ('app_race_5', '')} 

 {('app_race_4', '21'), ('app_race_5', '26'), ('app_race_2', '4'), ('app_race_3', '43')}
v635_4_const
v636_const
v637_const
{('co_app_race_3', ''), ('co_app_race_5', ''), ('co_app_race_4', ''), ('co_app_race_2', '')} 

 {('co_app_race_5', '43'), ('co_app_race_2', '3'), ('co_app_race_3', '22'), ('co_app_race_4', '27')}
v638_const
v639_const
v640_const
v641_const
v643_const
v644_const
v645_const
v647_const
v648_const
v6

v704_const
v705_const

starting constraints iteration 2
v610_const
v612_const
{('preapproval', '1')} 

 {('preapproval', '2')}
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_3', '12'), ('app_eth_4', '11'), ('app_eth_5', '13'), ('app_eth_2', '14')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')} 

 {('app_eth_3', '12'), ('app_eth_4', '11'), ('app_eth_5', '13'), ('app_eth_2', '14')}
v629_const
v630_const
{('co_app_eth_4', ''), ('co_app_eth_5', ''), ('co_app_eth_3', ''), ('co_app_eth_2', '')} 

 {('co_app_eth_3', '2'), ('co_app_eth_5', '11'), ('co_app_eth_4', '1'), ('co_app_eth_2', '13')}
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
v635_4_const
v636_const
v637_const
{('co_app_race_3', ''), ('co_app_race_5', ''), ('co_app_race_4


starting constraints iteration 8
v610_const
v612_const
{('preapproval', '1')} 

 {('preapproval', '2')}
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_3', '11'), ('app_eth_4', '13'), ('app_eth_2', '1'), ('app_eth_5', '12')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')} 

 {('app_eth_3', '11'), ('app_eth_4', '13'), ('app_eth_2', '1'), ('app_eth_5', '12')}
v629_const
v630_const
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
v635_4_const
v636_const
v637_const
v638_const
v639_const
v640_const
v641_const
v643_const
v644_const
v645_const
v647_const
v648_const
v649_const
v650_const
v651_const
v652_const
v654_const
v655_const
v656_const
v657_const
v658_const
v661_const
v662_const
v663_const
v664_const
v666_const
v667_const
v668_cons

...


starting constraints iteration 295
v610_const
v612_const
{('preapproval', '1')} 

 {('preapproval', '2')}
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_5', '2'), ('app_eth_3', '11'), ('app_eth_2', '14'), ('app_eth_4', '12')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_5', ''), ('app_eth_1', '2'), ('app_eth_2', ''), ('app_eth_3', ''), ('app_eth_4', '')} 

 {('app_eth_2', '14'), ('app_eth_5', '2'), ('app_eth_3', '11'), ('app_eth_1', '1'), ('app_eth_4', '12')}
v629_const
v630_const
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
v635_4_const
v636_const
v637_const
v638_const
v639_const
v640_const
v641_const
v643_const
v644_const
v645_const
v647_const
v648_const
v649_const
v650_const
v651_const
v652_const
v654_const
v655_const
v656_const
v657_const
v658_const
v661_const
v662_const
v663_const



starting constraints iteration 301
v610_const
v612_const
{('preapproval', '1')} 

 {('preapproval', '2')}
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_3', '14'), ('app_eth_2', '13'), ('app_eth_4', '1'), ('app_eth_5', '12')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')} 

 {('app_eth_3', '14'), ('app_eth_2', '13'), ('app_eth_4', '1'), ('app_eth_5', '12')}
v629_const
v630_const
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
v635_4_const
v636_const
v637_const
v638_const
v639_const
v640_const
v641_const
v643_const
v644_const
v645_const
v647_const
v648_const
v649_const
v650_const
v651_const
v652_const
v654_const
v655_const
v656_const
v657_const
v658_const
v661_const
v662_const
v663_const
v664_const
v666_const
v667_const
v668_co


starting constraints iteration 308
v610_const
v612_const
{('preapproval', '1')} 

 {('preapproval', '2')}
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_4', '11'), ('app_eth_3', '13'), ('app_eth_2', '12'), ('app_eth_5', '1')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_5', ''), ('app_eth_2', ''), ('app_eth_1', '1'), ('app_eth_3', ''), ('app_eth_4', '')} 

 {('app_eth_5', '1'), ('app_eth_3', '13'), ('app_eth_1', '2'), ('app_eth_2', '12'), ('app_eth_4', '11')}
v629_const
v630_const
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
v635_4_const
v636_const
v637_const
v638_const
v639_const
v640_const
v641_const
v643_const
v644_const
v645_const
v647_const
v648_const
v649_const
v650_const
v651_const
v652_const
v654_const
v655_const
v656_const
v657_const
v658_const
v661_const
v662_const
v663_const



starting constraints iteration 315
v610_const
v612_const
{('preapproval', '1')} 

 {('preapproval', '2')}
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_3', '2'), ('app_eth_4', '13'), ('app_eth_2', '14'), ('app_eth_5', '12')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_5', ''), ('app_eth_1', '2'), ('app_eth_2', ''), ('app_eth_3', ''), ('app_eth_4', '')} 

 {('app_eth_4', '13'), ('app_eth_2', '14'), ('app_eth_5', '12'), ('app_eth_3', '2'), ('app_eth_1', '1')}
v629_const
v630_const
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
v635_4_const
v636_const
v637_const
v638_const
v639_const
v640_const
v641_const
v643_const
v644_const
v645_const
v647_const
v648_const
v649_const
v650_const
v651_const
v652_const
v654_const
v655_const
v656_const
v657_const
v658_const
v661_const
v662_const
v663_const



starting constraints iteration 322
v610_const
v612_const
{('preapproval', '1')} 

 {('preapproval', '2')}
v613_2_const
v613_3_const
v613_4_const
{('preapproval', '2')} 

 {('preapproval', '1')}
v614_const
v615_const
v619_const
v622_const
v627_const
{('app_eth_3', '14'), ('app_eth_2', '13'), ('app_eth_4', '2'), ('app_eth_5', '12')} 

 {('app_eth_2', ''), ('app_eth_4', ''), ('app_eth_3', ''), ('app_eth_5', '')}
v628_const
{('app_eth_5', ''), ('app_eth_1', '2'), ('app_eth_2', ''), ('app_eth_3', ''), ('app_eth_4', '')} 

 {('app_eth_5', '12'), ('app_eth_3', '14'), ('app_eth_2', '13'), ('app_eth_1', '1'), ('app_eth_4', '2')}
v629_const
v630_const
v631_const
v632_const
v633_const
v634_const
v635_1_const
v635_2_const
v635_3_const
v635_4_const
v636_const
v637_const
v638_const
v639_const
v640_const
v641_const
v643_const
v644_const
v645_const
v647_const
v648_const
v649_const
v650_const
v651_const
v652_const
v654_const
v655_const
v656_const
v657_const
v658_const
v661_const
v662_const
v663_const


In [ ]:
#Quality and Macro field interrelationship constraints:

In [ ]:
#join LAR and TS rows to make an output file
def write_file(ts_input=None, lar_input=None, directory="../edits_files/", name="passes_all.txt"):
    """Takes a TS row as a dictionary and LAR data as a dataframe. Writes LAR data to file and 
    re-reads it to combine with TS data to make a full file."""
    #make directories for files if they do not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    #write LAR dataframe to file
    parts_dir = directory+"file_parts/"
    if not os.path.exists(parts_dir):
        os.makedirs(parts_dir)
        
    lar_input.to_csv(parts_dir + "lar_data.txt", sep="|", header=False, index=False, index_label=False)
    #load LAR data as file rows
    with open(parts_dir + "lar_data.txt", 'r') as lar_data:
        lar = lar_data.readlines()

    with open(directory + name, 'w') as final_file:
        final_file.write("|".join(ts_input.values())+"\n")
        for line in lar:
            final_file.write("{line}".format(line=line))

In [ ]:
#modify file for testing


In [ ]:
#write sample file to disk
write_file(ts_input=ts_row_small, lar_input=lar_frame, name="test.txt") #writes created file to disk
#validator engine uses the default: path="../edits_files/", data_file="passes_all.txt" for data files
validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties) #instantiate edits rules engine

In [ ]:
#split TS and LAR using validator function
#validator creates class objects of each of these internally as well
ts_df, lar_df = validator.split_ts_row()


In [ ]:
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(validator, func)()


In [ ]:
validator.results

In [ ]:
ulis=[]
validator.lar_df[["loan_purpose", "preapproval","action_taken","app_race_1", "app_race_2", "app_race_3", "app_race_4", "app_race_5"
                  ,"uli"]][validator.lar_df.uli.isin(ulis)]

In [ ]:
results_df = pd.DataFrame(validator.results)

In [ ]:
results_df[results_df.status=="failed"]

In [ ]:
validator.lar_df[["action_taken", "preapproval", "interest_rate","mlo_id","uli"]]

In [ ]:
set_row = set(row.items())
row_base["mlo_id"] = "17"
set_base = set(row_base.items())
diff = (set_row - set_base)
diff2 = (set_base - set_row)
diff, diff2

with open('/Users/roellk/Downloads/2016HMDALAR.csv') as f:
    for line in f.readlines():
        line = line.split(",")
        if line[1] in ("22-3039688", "223039688"):
            print(line)

with open('/Users/roellk/Downloads/2016HMDAReporterPanel.dat') as f:
    for line in f.readlines():
        if "22-3039688" in line or "223039688" in line:
            print(line)

In [ ]:
#ts_row_small
#row
#both ordered dicts
#columns=self.ts_field_names
#columns=self.lar_field_names

rules = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties)
ts_row_df = pd.DataFrame(ts_row_small, index=[0], columns=rules.ts_field_names)
lar_row_df = pd.DataFrame(row, index=[0], columns=rules.lar_field_names)
rules.load_ts_data(ts_row_df)
rules.load_lar_data(lar_row_df)

In [ ]:
for func in dir(rules):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        print("applying:", func)
        getattr(rules, func)()

In [ ]:
rules.results

In [ ]:
results_df = pd.DataFrame(rules.results)

In [ ]:
results_df[results_df.status=='failed']

In [ ]:
rules.lar_df[["preapproval", "action_taken"]]